In [1]:
import os
import datetime

import torch
import torch.multiprocessing as mp
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
from swadist.utils import spawn_fn

# mp.spawn may throw an error without this
os.environ['MKL_THREADING_LAYER'] = 'GNU'

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print('Using cuda')
else:
    print('Using cpu')

seed = int((datetime.date.today() - datetime.date(2022, 4, 11)).total_seconds())
print(f'seed: {seed}')

Using cuda
seed: 2246400


In [2]:
# number of model replicas
world_size = 2

dataloader_kwargs = {
    'dataset': 'cifar10',
    'batch_size': 256 // world_size,
    'num_workers': 2,
}

model_kwargs = {
    'n_classes': 10,
    'in_kernel_size': 3,
    'stack_sizes': [1, 1, 1],
    'batch_norm': False,
}

optimizer_kwargs = {
    'lr': 2**-5.,
    'momentum': 0.975,
    'nesterov': True,
}

trainer_kwargs = {
    # whether to log training to Tensorboard
    'log': False,
    'log_dir': './runs-notebooks',
}

train_kwargs = {
    'epochs_sgd': 0,
    'epochs_codist': 0,
    'epochs_swa': 15,
    'swadist': True,
    'codist_kwargs': {
        'sync_freq': 50,
        'transform': 'softmax',
        'debug': False,
    },
    'swadist_kwargs': {
        'max_averaged': 3,
        'sync_freq': 50,
        'transform': 'softmax',
        'debug': False,
    },
    # 'validations_per_epoch': 4,
    'save': False,
    'save_dir': './_state_dicts',
    # 'stopping_acc': 0.7,
}

scheduler_kwargs = {
    'alpha': 0.25,
    'decay_epochs': 15,
}

# swa_scheduler_kwargs = {
#     'swa_lr':  optimizer_kwargs['lr'] / 10, 
#     'anneal_strategy': 'cos',
#     'anneal_epochs': 3,
# }

In [3]:
%%time

trainer_kwargs['name'] = 'swadist-v2-data-parallel'

dataloader_kwargs['data_parallel'] = True
dataloader_kwargs['split_training'] = False

args = (world_size,
        dataloader_kwargs,
        model_kwargs,
        optimizer_kwargs,
        trainer_kwargs,
        train_kwargs,
        scheduler_kwargs,
        None, # swa_scheduler_kwargs,
        seed) # seed on rank i = seed + i

# begin training
mp.spawn(spawn_fn, args=args, nprocs=world_size, join=True)

Rank 0: joined process group on device cuda with backend nccl
Rank 0: torch.manual_seed(2246400)
Using DistributedSampler
Number of training samples: 45000
Number of training batches: 176


Worker 1/2 starting 15-epoch training loop...
SGD epochs: 0 | Codistillation epochs: 0 | SWADist epochs: 15
DistributedDataParallel: False
Stopping accuracy: None

Starting SWADist phase...

Rank 1: joined process group on device cuda with backend nccl
Rank 1: torch.manual_seed(2246400)
Using DistributedSampler
Number of training samples: 45000
Number of training batches: 176


Worker 2/2 starting 15-epoch training loop...
Train epoch: 1 | Metrics (epoch mean): cross_entropy=1.960304 <> acc=0.260868 <> swadist_loss=2.173560 | Batch (size 100): 176/176 (100%) | Total steps: 176
Rank 0 | Validation mean |  cross_entropy=2.014968 <> accuracy=0.277734 | Batch: 40/40 (100%)
Rank 1 | Validation mean |  cross_entropy=2.083179 <> accuracy=0.281250 | Batch: 40/40 (100%)

Train epoch: 2 | Metrics (epoch mean)